## Blockchain Gambling
---
* We created a blockchain based random number generator game. 
   > Participants will provide their ETH wallet address and enter a number between 1-100 as their guess. 
   > The closest guess to the randomly generated number wins.
* We also built a smart contract to withdraw your bet, in this case ETH, and store it in escrow until the game is completed and a winner is determined.
   > Once a winner is determined, the contract will deposit the pot into the winner's account.
* We used https://remix.ethereum.org/ to write our smart contract.

### Import libraries
---

In [1]:
# Import libraries
from web3 import Web3, HTTPProvider
import random

### Connect to Blockchain
---

In [2]:
# Ganache JSON RPC used to connect to Ethereum node: https://www.trufflesuite.com/ganache
endpoint = 'HTTP://127.0.0.1:7545'  

In [3]:
# Connect to smart contracct and blockchain with Web3.py
w3 = Web3(Web3.HTTPProvider(endpoint)) 
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.isConnected() 

True

### Smart Contract
---

> **ABI** Address changes when the code that is compiled changes. If the code base is the same, compiling will produce the same ABI. 
- ABI is generated once your smart contract is compiled.

> **Contract Addres** changes when we redeploy the contract.
* for the current address `0xA45ff7bd5BA79A3F8699743ED5B5d252B32135dB`

In [4]:
# Abstract Binary Interface
abi = [
	{
		"inputs": [],
		"name": "address_one",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "address_three",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "address_two",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "contractAddress",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "deposit",
		"outputs": [],
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_deposit_amount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_deposit_block",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_to_deposit",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_to_withdraw",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_withdraw_amount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "last_withdraw_block",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address payable",
				"name": "_one",
				"type": "address"
			},
			{
				"internalType": "address payable",
				"name": "_two",
				"type": "address"
			},
			{
				"internalType": "address payable",
				"name": "_three",
				"type": "address"
			}
		],
		"name": "players",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "prizeAmount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "winner",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address payable",
				"name": "_winner",
				"type": "address"
			}
		],
		"name": "withdraw",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"stateMutability": "payable",
		"type": "receive"
	}
]

In [5]:
# Create variable for contract address created from solidity contract
contract_address = '0xA45ff7bd5BA79A3F8699743ED5B5d252B32135dB'

# Connect to smart contract
WithdrawDeposit = w3.eth.contract(address=contract_address, abi=abi)

In [6]:
# Create players from local blockchain Ganache
players = [w3.eth.accounts[i+1]for i in range(3)]

# Display wallet addresses
players

['0x7D0d96c44dcbE750d3A9da13C442C226705B5815',
 '0xC6Bb775DF828Ea1aee61FeAa417b46bD991EF7F5',
 '0xD77501f48a3E4ca5410CcB8a5840EADACCE57aCf']

In [7]:
# Check on all available public functions within the smart contract
WithdrawDeposit.all_functions()

[<Function address_one()>,
 <Function address_three()>,
 <Function address_two()>,
 <Function contractAddress()>,
 <Function deposit()>,
 <Function getBalance()>,
 <Function last_deposit_amount()>,
 <Function last_deposit_block()>,
 <Function last_to_deposit()>,
 <Function last_to_withdraw()>,
 <Function last_withdraw_amount()>,
 <Function last_withdraw_block()>,
 <Function players(address,address,address)>,
 <Function prizeAmount()>,
 <Function winner()>,
 <Function withdraw(address)>]

### Creating our first transaction. 
* Use '*' to unpack entire list

In [8]:
# Assign wallet addresses to smart contract
WithdrawDeposit.functions.players(*players).transact({'from': '0x4ae123a8103AFA3d7b0023DE2CBeA75D24C80c40'})

HexBytes('0xc5851c4b6095f2a678cd7117733dee2809413cbac0048f0206768e613b8a7e74')

In [9]:
# Confirm players were written to the contract
WithdrawDeposit.functions.address_two().call()

'0xC6Bb775DF828Ea1aee61FeAa417b46bD991EF7F5'

In [10]:
# Making sure all players deposit 1 ETH into the contract
amount = 10**18 # This is the exponential number of wei make up 1 ETH
for p in players:
    WithdrawDeposit.functions.deposit().transact({'from':p, 'value': amount})

### Some validation functions
---

In [11]:
# Confirm all withdrawls are complete by displaying the smart contracts balance
WithdrawDeposit.functions.getBalance().call()

3000000000000000000

In [12]:
# Check who was the last deposit
WithdrawDeposit.functions.last_to_deposit().call()

'0xD77501f48a3E4ca5410CcB8a5840EADACCE57aCf'

In [13]:
# check last amount deposited
WithdrawDeposit.functions.last_deposit_amount().call()

1000000000000000000

In [14]:
# total Prize Amount
WithdrawDeposit.functions.prizeAmount().call()

3000000000000000000

### Random Number Genereator
---

In [15]:
# Generate random number from 1-100
random_number = random.randint(1,100)

# Input each players wallet address and guess
address_one = input('Enter your ETH wallet address - Bet Size is 1 ETH')
player_one = input('Enter a number between 1 and 100')

address_two = input('Enter your ETH wallet address - Bet Size is 1 ETH')
player_two = input('Enter a number between 1 and 100')

address_three =input('Enter your ETH wallet address - Bet Size is 1 ETH')
player_three = input('Enter a number between 1 and 100')

# Convert guesses from string to integer
one_num = int(player_one)
two_num = int(player_two)
three_num = int(player_three)

# list player guess integers
guesses = [one_num, two_num, three_num]

# Choose the closest guess to the random number
winner = min(guesses, key=lambda x: abs(x - random_number))

# Print the random number and the winner
print(f'The random number is {random_number}')
print(f'Winner Winner Chicken Dinner! {winner} is the winning number!')

Enter your ETH wallet address - Bet Size is 1 ETH 0x7D0d96c44dcbE750d3A9da13C442C226705B5815
Enter a number between 1 and 100 45
Enter your ETH wallet address - Bet Size is 1 ETH 0xC6Bb775DF828Ea1aee61FeAa417b46bD991EF7F5
Enter a number between 1 and 100 64
Enter your ETH wallet address - Bet Size is 1 ETH 0xD77501f48a3E4ca5410CcB8a5840EADACCE57aCf
Enter a number between 1 and 100 12


The random number is 56
Winner Winner Chicken Dinner! 64 is the winning number!


In [16]:
# Make variables for won function below
w1 = (address_one, one_num)
w2 = (address_two, two_num)
w3 = (address_three, three_num)

In [28]:
# determine winning players wallet address
def won(p1, p2, p3, winner):
    '''
    note: maximum three players
    Arguemnts: p1 tuple for wallet address and quess number for player one
    '''
    if winner == p1[1]:
        winner = p1[0]
    elif winner == p2[1]:
        winner = p2[0]
    elif winner == p3[1]:
        winner = p3[0]
    return winner

win = won(w1, w2, w3, winner)

In [29]:
# Display winning wallet address
win

'0xC6Bb775DF828Ea1aee61FeAa417b46bD991EF7F5'

### Complete the smart contract
---

In [22]:
# Pay the winner
WithdrawDeposit.functions.withdraw(win).transact({'from': '0x4ae123a8103AFA3d7b0023DE2CBeA75D24C80c40'})

HexBytes('0x11d2d720ae8d5f32f2f9cc755f9bc21ea1e2044a03c97ed9ad7b458464294f32')

In [23]:
# Display amount withdrawn from the smart contract
WithdrawDeposit.functions.last_withdraw_amount().call()

3000000000000000000

In [24]:
# Display wallet address of last withdraw (Winning Address)
WithdrawDeposit.functions.last_to_withdraw().call()

'0xC6Bb775DF828Ea1aee61FeAa417b46bD991EF7F5'

In [30]:
# Compare wallet address from winner and where funds were sent
ww = WithdrawDeposit.functions.last_to_withdraw().call()
ww == win

True

In [25]:
# Confrim money is no longer in smart contract
WithdrawDeposit.functions.getBalance().call()

0

In [26]:
# Display current amount in prize money function within smart contract
WithdrawDeposit.functions.prizeAmount().call()

0

### Solidity Code
---
https://remix.ethereum.org 

In [ ]:
pragma solidity >=0.6.2 <0.8.0;

contract WithdrawDeposit {
    
    address payable public address_one;
    address payable public address_two;
    address payable public address_three;
    address payable public winner;
    
    address public last_to_withdraw;
    uint public last_withdraw_block;
    uint public last_withdraw_amount;
  
    address public last_to_deposit;
    uint public last_deposit_block;
    uint public last_deposit_amount;
    
    uint public prizeAmount = 0;
    address public contractAddress = address(this);
    
    
    function players(address payable _one, address payable _two, address payable _three) external {
        
        address_one = _one;
        address_two = _two;
        address_three = _three;
    
    }
    
    function withdraw(address payable _winner) public {
        require(_winner == address_one || _winner == address_two || _winner == address_three, "Not a Participant!");
        
        last_to_withdraw = _winner;
        last_withdraw_block = block.number;
        last_withdraw_amount = prizeAmount;
        
        _winner.transfer(prizeAmount);  
         
        prizeAmount=0;
    
  }
  
    function getBalance() public view returns(uint) {
        return address(this).balance;
  }